# Import lib

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
%matplotlib inline

data_train = pd.read_csv('/home/dan/Test/Kaggle/data/Titanic/train.csv', index_col='PassengerId')
data_test = pd.read_csv('/home/dan/Test/Kaggle/data/Titanic/test.csv', index_col='PassengerId')

data_train.head(3)

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


In [9]:
y_train = data_train['Survived']
data_train = data_train.drop(['Survived'], axis=1) 

# Features importance

In [ ]:
'''
Survival - выжил
SibSp - братьев и сестер на борту
parch - родителей детей на борту
fare - тариф
Embarked - порт погрузки C = Cherbourg, Q = Queenstown, S = Southampton'''

In [ ]:
sns.barplot(x="Embarked", y="Survived", hue="Parch", data=data_train);

In [ ]:
sns.pointplot(x="Pclass", y="Survived", hue="Sex", data=data_train,
              palette={"male": "blue", "female": "pink"},
              markers=["*", "o"], linestyles=["-", "--"]);


In [ ]:
col = data_train.columns
for column in col:
    print(column)
    print(data_train[column].value_counts())
    print('-'*40)

In [ ]:
pd.crosstab(data_train['Survived'], data_train['Parch'])

In [ ]:
pd.crosstab(data_train['Survived'], data_train['Cabin'])

In [ ]:
pd.crosstab(data_train['Survived'], data_train['Sex'])

# Preprocessing

## Change Feature

In [2]:
def simplify_ages(df):
    df.Age = df.Age.fillna(-0.5)
    bins = (-1, 0, 5, 12, 18, 25, 35, 60, 120)
    group_names = ['Unknown', 'Baby', 'Child', 'Teenager', 'Student', 'Young Adult', 'Adult', 'Senior']
    categories = pd.cut(df.Age, bins, labels=group_names)
    df.Age = categories
    return df
def simplify_cabins(df):
    df.Cabin = df.Cabin.fillna('N')
    df.Cabin = df.Cabin.apply(lambda x: x[0])
    return df
def simplify_fares(df):
    df.Fare = df.Fare.fillna(-0.5)
    bins = (-1, 0, 8, 15, 31, 1000)
    group_names = ['Unknown', '1_quartile', '2_quartile', '3_quartile', '4_quartile']
    categories = pd.cut(df.Fare, bins, labels=group_names)
    df.Fare = categories
    return df
def drop_features(df):
    return df.drop(['Ticket', 'Name', 'Embarked'], axis=1)
def transform_features(df):
    df = simplify_ages(df)
    df = simplify_cabins(df)
    df = simplify_fares(df)
    df = drop_features(df)
    return df

In [3]:
data_train = transform_features(data_train)
data_test = transform_features(data_test)

In [4]:
data_train['Parch_Cabin_Sex'] = ((data_train['Parch'] == 0) & \
                                 (data_train['Cabin'] == 'N') & \
                                 (data_train['Sex'] == 'female')).astype('int')
data_test['Parch_Cabin_Sex'] = ((data_test['Parch'] == 0) & \
                                 (data_test['Cabin'] == 'N') & \
                                 (data_test['Sex'] == 'female')).astype('int') 

## LabelEncoder

In [5]:
def LE(df_train,df_test):
    le = preprocessing.LabelEncoder()
    features = ['Fare','Sex', 'Age', 'Cabin']
    df_combined = pd.concat([df_train[features], df_test[features]])
    for feature in features:
        le = le.fit(df_combined[feature])
        df_train[feature] = le.transform(df_train[feature])
        df_test[feature] = le.transform(df_test[feature])
    return  df_train, df_test

In [6]:
data_train, data_test = LE(data_train,data_test)
data_train.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Parch_Cabin_Sex
PassengerId,,,,,,,,,
1,0,3,1,4,1,0,0,7,0
2,1,1,0,0,1,0,3,2,0
3,1,3,0,7,0,0,0,7,1
4,1,1,0,7,1,0,3,2,0
5,0,3,1,7,0,0,1,7,0


## StandartScaler

In [7]:
from sklearn.preprocessing import StandardScaler
def to_scaler(df_train, df_test):    
    scaler = StandardScaler()
    features = ['Age', 'Cabin', 'Pclass']
    for feature in features:
        data_train[feature] = scaler.fit_transform(data_train[feature].values.reshape(-1, 1))
        data_test[feature] = scaler.transform(data_test[feature].values.reshape(-1, 1))
    return df_train, df_test

In [8]:
data_train, data_test = to_scaler(data_train, data_test);
data_train.head()

/home/dan/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Parch_Cabin_Sex
PassengerId,,,,,,,,,
1,0,0.827377,1,-0.011933,1,0,0,0.511293,0
2,1,-1.566107,0,-1.530855,1,0,3,-1.914491,0
3,1,0.827377,0,1.127258,0,0,0,0.511293,1
4,1,-1.566107,0,1.127258,1,0,3,-1.914491,0
5,0,0.827377,1,1.127258,0,0,1,0.511293,0


# Models

In [ ]:
data_test.head()

In [ ]:
from sklearn.model_selection import train_test_split
size_test = 0.20
X_train, X_valid, y_train, y_valid = train_test_split(data_train, y_train, test_size=size_test, random_state=17)

## SVM

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
acc_scorer = make_scorer(accuracy_score)

In [ ]:
from sklearn.svm import LinearSVC 

In [ ]:
clf_SVC = LinearSVC(random_state=17)

In [ ]:
parameters = {'C': [0.0001, 0.001, 0.01, 0.1], 
              'loss': ['squared_hinge', 'hinge' ] 
             }

In [ ]:
grid_obj = GridSearchCV(clf_SVC, parameters, scoring=acc_scorer)
grid_obj = grid_obj.fit(X_train, y_train)

In [ ]:
SVC_best = grid_obj.best_estimator_

In [ ]:
clf_SVC.fit(X_train, y_train)

In [ ]:
y_predictions = clf_SVC.predict(X_valid)
print(accuracy_score(y_valid, y_predictions)) 

## SGD

In [ ]:
from sklearn.linear_model import SGDClassifier

In [ ]:
clf_sgd = SGDClassifier(random_state = 17, max_iter = 1000, tol = 1e-4)

In [ ]:
parameters = {'loss': [ 'hinge', 'log', 'squared_hinge', 'squared_loss', 'huber'], 
              'penalty': ['l1', 'l2','elasticnet'], 
             }

In [ ]:
grid_obj = GridSearchCV(clf_sgd, parameters, scoring=acc_scorer)
grid_obj = grid_obj.fit(X_train, y_train)

In [ ]:
SGD_best = grid_obj.best_estimator_

In [ ]:
SGD_best.fit(X_train, y_train)

In [ ]:
y_predictions = SGD_best.predict(X_valid)
print(accuracy_score(y_valid, y_predictions)) 

## DecisionTreeClassifier

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
clf_DTC = DecisionTreeClassifier(random_state = 17)

In [ ]:
parameters = {'criterion': [ 'gini', 'entropy'], 
              'max_depth': list(np.arange(1,10,2)),
              'min_samples_leaf': list(np.arange(1,4,1))
             }

In [ ]:
grid_obj = GridSearchCV(clf_DTC, parameters, scoring=acc_scorer)
grid_obj = grid_obj.fit(X_train, y_train)

In [ ]:
DTC_best = grid_obj.best_estimator_

In [ ]:
clf_DTC.fit(X_train, y_train)

In [ ]:
y_predictions = clf_DTC.predict(X_valid)
print(accuracy_score(y_valid, y_predictions)) 

## RandomForestClassifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
clf_RFC = RandomForestClassifier(random_state=17)

In [ ]:
clf_RFC.fit(X_train, y_train)

In [ ]:
y_predictions = clf_RFC.predict(X_valid)
print(accuracy_score(y_valid, y_predictions)) 

## GaussianNB

In [ ]:
from sklearn.naive_bayes import GaussianNB
clf_gnb = GaussianNB()
y_pred = clf_gnb.fit(X_train, y_train).predict(X_valid)

In [ ]:
print(accuracy_score(y_valid, y_pred)) 

## CatBoostClassifier

In [34]:
from catboost import CatBoostClassifier
from sklearn.model_selection import cross_val_score,train_test_split, StratifiedKFold
from sklearn.metrics import roc_auc_score

In [11]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
clf_CTB = CatBoostClassifier(random_state=17)

In [12]:
train_df_part, valid_df, y_train_part, y_valid = train_test_split(data_train, y_train, 
                                                                  test_size=.2, stratify=y_train, 
                                                                  random_state=17)

In [29]:
train_df_part.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Parch_Cabin_Sex
PassengerId,,,,,,,,
878,0.827377,1,-0.011933,0,0,0,0.511293,0
797,-1.566107,0,-1.530855,0,0,2,-1.429334,0
737,0.827377,0,-1.530855,1,3,3,0.511293,0
605,-1.566107,1,1.127258,0,0,2,0.511293,0
558,-1.566107,1,0.747528,0,0,3,0.511293,0


In [32]:
cat_features_idx = np.where(train_df_part.dtypes == 'int')[0].tolist()
cat_features_idx 

[1, 3, 4, 5, 7]

In [35]:
%%time
cv_scores = []
for train_idx, test_idx in skf.split(train_df_part, y_train_part):
    cv_train_df, cv_valid_df = train_df_part.iloc[train_idx, :], train_df_part.iloc[test_idx, :]
    y_cv_train, y_cv_valid = y_train_part.iloc[train_idx], y_train_part.iloc[test_idx]
    
    clf_CTB.fit(cv_train_df, y_cv_train,
        cat_features=cat_features_idx);
    
    cv_scores.append(roc_auc_score(y_cv_valid, clf_CTB.predict_proba(cv_valid_df)[:, 1]))

0:	learn: 0.6824076	total: 3.13ms	remaining: 3.13s
1:	learn: 0.6650513	total: 10.3ms	remaining: 5.15s
2:	learn: 0.6496256	total: 15.5ms	remaining: 5.14s
3:	learn: 0.6359485	total: 20.4ms	remaining: 5.08s
4:	learn: 0.6221827	total: 25.7ms	remaining: 5.12s
5:	learn: 0.6106216	total: 30.7ms	remaining: 5.08s
6:	learn: 0.5999002	total: 35ms	remaining: 4.97s
7:	learn: 0.5883212	total: 42.6ms	remaining: 5.28s
8:	learn: 0.5784298	total: 49.7ms	remaining: 5.48s
9:	learn: 0.5698751	total: 55.2ms	remaining: 5.46s
10:	learn: 0.5645942	total: 67.3ms	remaining: 6.05s
11:	learn: 0.5598460	total: 71.5ms	remaining: 5.89s
12:	learn: 0.5519431	total: 80.9ms	remaining: 6.14s
13:	learn: 0.5459455	total: 86.2ms	remaining: 6.07s
14:	learn: 0.5387867	total: 92.6ms	remaining: 6.08s
15:	learn: 0.5328847	total: 98ms	remaining: 6.03s
16:	learn: 0.5273428	total: 103ms	remaining: 5.98s
17:	learn: 0.5226483	total: 118ms	remaining: 6.46s
18:	learn: 0.5196644	total: 127ms	remaining: 6.56s
19:	learn: 0.5169242	total: 1

182:	learn: 0.3798490	total: 984ms	remaining: 4.39s
183:	learn: 0.3797579	total: 993ms	remaining: 4.4s
184:	learn: 0.3795220	total: 1000ms	remaining: 4.4s
185:	learn: 0.3791700	total: 1.01s	remaining: 4.41s
186:	learn: 0.3787142	total: 1.01s	remaining: 4.41s
187:	learn: 0.3787062	total: 1.02s	remaining: 4.4s
188:	learn: 0.3781233	total: 1.02s	remaining: 4.4s
189:	learn: 0.3777386	total: 1.04s	remaining: 4.43s
190:	learn: 0.3773997	total: 1.04s	remaining: 4.43s
191:	learn: 0.3772725	total: 1.05s	remaining: 4.42s
192:	learn: 0.3771679	total: 1.06s	remaining: 4.42s
193:	learn: 0.3767013	total: 1.06s	remaining: 4.42s
194:	learn: 0.3765065	total: 1.07s	remaining: 4.42s
195:	learn: 0.3764568	total: 1.07s	remaining: 4.41s
196:	learn: 0.3761377	total: 1.08s	remaining: 4.4s
197:	learn: 0.3758551	total: 1.08s	remaining: 4.4s
198:	learn: 0.3749648	total: 1.09s	remaining: 4.39s
199:	learn: 0.3749648	total: 1.09s	remaining: 4.37s
200:	learn: 0.3741345	total: 1.1s	remaining: 4.37s
201:	learn: 0.3740

354:	learn: 0.3102636	total: 1.99s	remaining: 3.61s
355:	learn: 0.3096306	total: 2s	remaining: 3.61s
356:	learn: 0.3088855	total: 2s	remaining: 3.61s
357:	learn: 0.3086155	total: 2.01s	remaining: 3.61s
358:	learn: 0.3084409	total: 2.03s	remaining: 3.62s
359:	learn: 0.3084408	total: 2.03s	remaining: 3.61s
360:	learn: 0.3084408	total: 2.03s	remaining: 3.6s
361:	learn: 0.3082832	total: 2.04s	remaining: 3.6s
362:	learn: 0.3075727	total: 2.05s	remaining: 3.6s
363:	learn: 0.3075727	total: 2.05s	remaining: 3.59s
364:	learn: 0.3073099	total: 2.06s	remaining: 3.58s
365:	learn: 0.3067764	total: 2.07s	remaining: 3.58s
366:	learn: 0.3065640	total: 2.07s	remaining: 3.58s
367:	learn: 0.3062825	total: 2.08s	remaining: 3.57s
368:	learn: 0.3060901	total: 2.09s	remaining: 3.57s
369:	learn: 0.3060901	total: 2.09s	remaining: 3.56s
370:	learn: 0.3059556	total: 2.1s	remaining: 3.55s
371:	learn: 0.3054856	total: 2.1s	remaining: 3.55s
372:	learn: 0.3046797	total: 2.11s	remaining: 3.54s
373:	learn: 0.3044764	t

522:	learn: 0.2669869	total: 2.98s	remaining: 2.72s
523:	learn: 0.2669305	total: 2.99s	remaining: 2.71s
524:	learn: 0.2666252	total: 3s	remaining: 2.71s
525:	learn: 0.2663597	total: 3.01s	remaining: 2.71s
526:	learn: 0.2659609	total: 3.02s	remaining: 2.71s
527:	learn: 0.2657882	total: 3.02s	remaining: 2.7s
528:	learn: 0.2655936	total: 3.03s	remaining: 2.7s
529:	learn: 0.2654437	total: 3.03s	remaining: 2.69s
530:	learn: 0.2653167	total: 3.04s	remaining: 2.68s
531:	learn: 0.2651774	total: 3.04s	remaining: 2.68s
532:	learn: 0.2648402	total: 3.05s	remaining: 2.67s
533:	learn: 0.2644832	total: 3.05s	remaining: 2.67s
534:	learn: 0.2642592	total: 3.06s	remaining: 2.66s
535:	learn: 0.2640590	total: 3.06s	remaining: 2.65s
536:	learn: 0.2639589	total: 3.07s	remaining: 2.65s
537:	learn: 0.2637710	total: 3.07s	remaining: 2.64s
538:	learn: 0.2635805	total: 3.08s	remaining: 2.63s
539:	learn: 0.2633988	total: 3.08s	remaining: 2.63s
540:	learn: 0.2631147	total: 3.09s	remaining: 2.62s
541:	learn: 0.262

694:	learn: 0.2340765	total: 3.96s	remaining: 1.74s
695:	learn: 0.2340182	total: 3.97s	remaining: 1.73s
696:	learn: 0.2338839	total: 3.98s	remaining: 1.73s
697:	learn: 0.2338505	total: 3.98s	remaining: 1.72s
698:	learn: 0.2336700	total: 3.99s	remaining: 1.72s
699:	learn: 0.2333924	total: 4s	remaining: 1.71s
700:	learn: 0.2332843	total: 4s	remaining: 1.71s
701:	learn: 0.2331259	total: 4.01s	remaining: 1.7s
702:	learn: 0.2330221	total: 4.02s	remaining: 1.7s
703:	learn: 0.2328536	total: 4.02s	remaining: 1.69s
704:	learn: 0.2327512	total: 4.03s	remaining: 1.68s
705:	learn: 0.2326683	total: 4.03s	remaining: 1.68s
706:	learn: 0.2325745	total: 4.04s	remaining: 1.67s
707:	learn: 0.2322456	total: 4.04s	remaining: 1.67s
708:	learn: 0.2321762	total: 4.05s	remaining: 1.66s
709:	learn: 0.2319318	total: 4.05s	remaining: 1.66s
710:	learn: 0.2317634	total: 4.06s	remaining: 1.65s
711:	learn: 0.2316602	total: 4.06s	remaining: 1.64s
712:	learn: 0.2316071	total: 4.07s	remaining: 1.64s
713:	learn: 0.231483

...

206:	learn: 0.3515111	total: 999ms	remaining: 3.83s
207:	learn: 0.3513376	total: 1s	remaining: 3.83s
208:	learn: 0.3508801	total: 1.01s	remaining: 3.83s
209:	learn: 0.3507232	total: 1.02s	remaining: 3.83s
210:	learn: 0.3500150	total: 1.03s	remaining: 3.84s
211:	learn: 0.3498971	total: 1.03s	remaining: 3.85s
212:	learn: 0.3495662	total: 1.04s	remaining: 3.85s
213:	learn: 0.3492709	total: 1.05s	remaining: 3.85s
214:	learn: 0.3488609	total: 1.05s	remaining: 3.84s
215:	learn: 0.3488609	total: 1.06s	remaining: 3.84s
216:	learn: 0.3486740	total: 1.06s	remaining: 3.84s
217:	learn: 0.3483892	total: 1.07s	remaining: 3.84s
218:	learn: 0.3479363	total: 1.08s	remaining: 3.84s
219:	learn: 0.3474539	total: 1.08s	remaining: 3.84s
220:	learn: 0.3469626	total: 1.09s	remaining: 3.84s
221:	learn: 0.3464111	total: 1.09s	remaining: 3.83s
222:	learn: 0.3460181	total: 1.1s	remaining: 3.83s
223:	learn: 0.3454010	total: 1.1s	remaining: 3.82s
224:	learn: 0.3453366	total: 1.11s	remaining: 3.82s
225:	learn: 0.345

380:	learn: 0.2902248	total: 1.98s	remaining: 3.22s
381:	learn: 0.2898428	total: 1.99s	remaining: 3.22s
382:	learn: 0.2889183	total: 2s	remaining: 3.22s
383:	learn: 0.2886195	total: 2.01s	remaining: 3.22s
384:	learn: 0.2883517	total: 2.02s	remaining: 3.23s
385:	learn: 0.2880401	total: 2.03s	remaining: 3.23s
386:	learn: 0.2879461	total: 2.03s	remaining: 3.22s
387:	learn: 0.2878288	total: 2.04s	remaining: 3.22s
388:	learn: 0.2869747	total: 2.04s	remaining: 3.21s
389:	learn: 0.2868810	total: 2.05s	remaining: 3.21s
390:	learn: 0.2867952	total: 2.06s	remaining: 3.2s
391:	learn: 0.2864023	total: 2.06s	remaining: 3.2s
392:	learn: 0.2860300	total: 2.07s	remaining: 3.19s
393:	learn: 0.2857780	total: 2.07s	remaining: 3.19s
394:	learn: 0.2855838	total: 2.08s	remaining: 3.18s
395:	learn: 0.2854247	total: 2.08s	remaining: 3.18s
396:	learn: 0.2853401	total: 2.09s	remaining: 3.18s
397:	learn: 0.2850493	total: 2.1s	remaining: 3.17s
398:	learn: 0.2848687	total: 2.1s	remaining: 3.17s
399:	learn: 0.28461

552:	learn: 0.2491662	total: 2.98s	remaining: 2.41s
553:	learn: 0.2490730	total: 2.99s	remaining: 2.4s
554:	learn: 0.2489562	total: 3s	remaining: 2.4s
555:	learn: 0.2487970	total: 3s	remaining: 2.4s
556:	learn: 0.2487018	total: 3.02s	remaining: 2.4s
557:	learn: 0.2485488	total: 3.02s	remaining: 2.39s
558:	learn: 0.2484830	total: 3.03s	remaining: 2.39s
559:	learn: 0.2483286	total: 3.04s	remaining: 2.38s
560:	learn: 0.2480375	total: 3.04s	remaining: 2.38s
561:	learn: 0.2479666	total: 3.05s	remaining: 2.37s
562:	learn: 0.2478900	total: 3.05s	remaining: 2.37s
563:	learn: 0.2478246	total: 3.06s	remaining: 2.36s
564:	learn: 0.2473569	total: 3.06s	remaining: 2.36s
565:	learn: 0.2470933	total: 3.07s	remaining: 2.35s
566:	learn: 0.2466355	total: 3.07s	remaining: 2.35s
567:	learn: 0.2466040	total: 3.08s	remaining: 2.34s
568:	learn: 0.2463964	total: 3.08s	remaining: 2.34s
569:	learn: 0.2463446	total: 3.09s	remaining: 2.33s
570:	learn: 0.2463140	total: 3.09s	remaining: 2.32s
571:	learn: 0.2460998	

721:	learn: 0.2244019	total: 3.98s	remaining: 1.53s
722:	learn: 0.2242148	total: 4s	remaining: 1.53s
723:	learn: 0.2241921	total: 4.01s	remaining: 1.53s
724:	learn: 0.2241569	total: 4.02s	remaining: 1.52s
725:	learn: 0.2241270	total: 4.03s	remaining: 1.52s
726:	learn: 0.2240578	total: 4.03s	remaining: 1.51s
727:	learn: 0.2239604	total: 4.04s	remaining: 1.51s
728:	learn: 0.2238420	total: 4.04s	remaining: 1.5s
729:	learn: 0.2238056	total: 4.05s	remaining: 1.5s
730:	learn: 0.2237332	total: 4.05s	remaining: 1.49s
731:	learn: 0.2234634	total: 4.05s	remaining: 1.48s
732:	learn: 0.2232754	total: 4.06s	remaining: 1.48s
733:	learn: 0.2232389	total: 4.07s	remaining: 1.47s
734:	learn: 0.2231562	total: 4.07s	remaining: 1.47s
735:	learn: 0.2231069	total: 4.08s	remaining: 1.46s
736:	learn: 0.2229817	total: 4.08s	remaining: 1.46s
737:	learn: 0.2229194	total: 4.09s	remaining: 1.45s
738:	learn: 0.2228775	total: 4.09s	remaining: 1.45s
739:	learn: 0.2227644	total: 4.1s	remaining: 1.44s
740:	learn: 0.2224

893:	learn: 0.2086851	total: 4.97s	remaining: 589ms
894:	learn: 0.2086131	total: 4.98s	remaining: 584ms
895:	learn: 0.2085676	total: 4.98s	remaining: 578ms
896:	learn: 0.2085554	total: 4.99s	remaining: 573ms
897:	learn: 0.2084755	total: 5s	remaining: 568ms
898:	learn: 0.2083779	total: 5.01s	remaining: 562ms
899:	learn: 0.2083078	total: 5.01s	remaining: 557ms
900:	learn: 0.2082662	total: 5.02s	remaining: 551ms
901:	learn: 0.2082202	total: 5.02s	remaining: 546ms
902:	learn: 0.2081930	total: 5.03s	remaining: 540ms
903:	learn: 0.2081173	total: 5.03s	remaining: 534ms
904:	learn: 0.2078012	total: 5.04s	remaining: 529ms
905:	learn: 0.2077777	total: 5.04s	remaining: 523ms
906:	learn: 0.2075689	total: 5.05s	remaining: 518ms
907:	learn: 0.2075106	total: 5.05s	remaining: 512ms
908:	learn: 0.2073360	total: 5.06s	remaining: 507ms
909:	learn: 0.2073273	total: 5.07s	remaining: 501ms
910:	learn: 0.2072371	total: 5.07s	remaining: 495ms
911:	learn: 0.2071475	total: 5.08s	remaining: 490ms
912:	learn: 0.2

In [36]:
np.mean(cv_scores)

0.86341318092101782

In [37]:
%%time
clf_CTB.fit(train_df_part, y_train_part,
        cat_features=cat_features_idx);

0:	learn: 0.6742021	total: 7.49ms	remaining: 7.48s
1:	learn: 0.6572037	total: 13.5ms	remaining: 6.73s
2:	learn: 0.6423374	total: 19.4ms	remaining: 6.46s
3:	learn: 0.6295784	total: 25.2ms	remaining: 6.28s
4:	learn: 0.6148649	total: 37.5ms	remaining: 7.46s
5:	learn: 0.6029177	total: 44.9ms	remaining: 7.44s
6:	learn: 0.5929009	total: 51.5ms	remaining: 7.3s
7:	learn: 0.5813195	total: 59.5ms	remaining: 7.37s
8:	learn: 0.5708356	total: 75.2ms	remaining: 8.28s
9:	learn: 0.5652719	total: 82ms	remaining: 8.12s
10:	learn: 0.5589646	total: 87.5ms	remaining: 7.86s
11:	learn: 0.5538672	total: 93.5ms	remaining: 7.7s
12:	learn: 0.5452745	total: 102ms	remaining: 7.77s
13:	learn: 0.5372527	total: 122ms	remaining: 8.59s
14:	learn: 0.5296226	total: 139ms	remaining: 9.11s
15:	learn: 0.5236178	total: 145ms	remaining: 8.94s
16:	learn: 0.5199137	total: 152ms	remaining: 8.79s
17:	learn: 0.5136652	total: 158ms	remaining: 8.62s
18:	learn: 0.5107855	total: 161ms	remaining: 8.31s
19:	learn: 0.5053320	total: 167ms

175:	learn: 0.3681682	total: 1.2s	remaining: 5.6s
176:	learn: 0.3679834	total: 1.21s	remaining: 5.61s
177:	learn: 0.3679551	total: 1.21s	remaining: 5.6s
178:	learn: 0.3678915	total: 1.22s	remaining: 5.6s
179:	learn: 0.3670009	total: 1.23s	remaining: 5.61s
180:	learn: 0.3658193	total: 1.24s	remaining: 5.62s
181:	learn: 0.3656336	total: 1.25s	remaining: 5.61s
182:	learn: 0.3655316	total: 1.25s	remaining: 5.6s
183:	learn: 0.3653724	total: 1.26s	remaining: 5.58s
184:	learn: 0.3645711	total: 1.26s	remaining: 5.57s
185:	learn: 0.3642517	total: 1.27s	remaining: 5.56s
186:	learn: 0.3639580	total: 1.28s	remaining: 5.55s
187:	learn: 0.3635842	total: 1.28s	remaining: 5.54s
188:	learn: 0.3629882	total: 1.29s	remaining: 5.53s
189:	learn: 0.3629881	total: 1.29s	remaining: 5.51s
190:	learn: 0.3625753	total: 1.3s	remaining: 5.51s
191:	learn: 0.3621248	total: 1.31s	remaining: 5.5s
192:	learn: 0.3619799	total: 1.31s	remaining: 5.5s
193:	learn: 0.3613449	total: 1.32s	remaining: 5.49s
194:	learn: 0.361064

342:	learn: 0.3130790	total: 2.42s	remaining: 4.63s
343:	learn: 0.3128919	total: 2.42s	remaining: 4.63s
344:	learn: 0.3123656	total: 2.44s	remaining: 4.63s
345:	learn: 0.3120414	total: 2.45s	remaining: 4.62s
346:	learn: 0.3119469	total: 2.46s	remaining: 4.62s
347:	learn: 0.3118495	total: 2.46s	remaining: 4.62s
348:	learn: 0.3117321	total: 2.47s	remaining: 4.61s
349:	learn: 0.3110224	total: 2.48s	remaining: 4.6s
350:	learn: 0.3108054	total: 2.48s	remaining: 4.59s
351:	learn: 0.3105536	total: 2.49s	remaining: 4.59s
352:	learn: 0.3103595	total: 2.5s	remaining: 4.58s
353:	learn: 0.3097859	total: 2.5s	remaining: 4.57s
354:	learn: 0.3093476	total: 2.51s	remaining: 4.56s
355:	learn: 0.3089917	total: 2.52s	remaining: 4.55s
356:	learn: 0.3084778	total: 2.52s	remaining: 4.54s
357:	learn: 0.3083536	total: 2.53s	remaining: 4.53s
358:	learn: 0.3082794	total: 2.53s	remaining: 4.52s
359:	learn: 0.3079758	total: 2.54s	remaining: 4.51s
360:	learn: 0.3078234	total: 2.54s	remaining: 4.5s
361:	learn: 0.30

524:	learn: 0.2734486	total: 3.62s	remaining: 3.27s
525:	learn: 0.2733204	total: 3.63s	remaining: 3.27s
526:	learn: 0.2731934	total: 3.64s	remaining: 3.27s
527:	learn: 0.2730901	total: 3.65s	remaining: 3.26s
528:	learn: 0.2728739	total: 3.66s	remaining: 3.26s
529:	learn: 0.2728177	total: 3.67s	remaining: 3.25s
530:	learn: 0.2726658	total: 3.67s	remaining: 3.24s
531:	learn: 0.2725236	total: 3.68s	remaining: 3.23s
532:	learn: 0.2723324	total: 3.68s	remaining: 3.23s
533:	learn: 0.2722198	total: 3.69s	remaining: 3.22s
534:	learn: 0.2721686	total: 3.7s	remaining: 3.21s
535:	learn: 0.2720617	total: 3.71s	remaining: 3.21s
536:	learn: 0.2717700	total: 3.72s	remaining: 3.21s
537:	learn: 0.2717267	total: 3.74s	remaining: 3.21s
538:	learn: 0.2716309	total: 3.76s	remaining: 3.21s
539:	learn: 0.2715807	total: 3.77s	remaining: 3.21s
540:	learn: 0.2715194	total: 3.78s	remaining: 3.21s
541:	learn: 0.2712229	total: 3.8s	remaining: 3.21s
542:	learn: 0.2711390	total: 3.82s	remaining: 3.22s
543:	learn: 0.

694:	learn: 0.2485670	total: 5.26s	remaining: 2.31s
695:	learn: 0.2485227	total: 5.27s	remaining: 2.3s
696:	learn: 0.2484986	total: 5.28s	remaining: 2.29s
697:	learn: 0.2482732	total: 5.3s	remaining: 2.29s
698:	learn: 0.2481763	total: 5.3s	remaining: 2.28s
699:	learn: 0.2480367	total: 5.31s	remaining: 2.28s
700:	learn: 0.2478896	total: 5.32s	remaining: 2.27s
701:	learn: 0.2477843	total: 5.33s	remaining: 2.26s
702:	learn: 0.2476874	total: 5.33s	remaining: 2.25s
703:	learn: 0.2474087	total: 5.34s	remaining: 2.25s
704:	learn: 0.2472253	total: 5.35s	remaining: 2.24s
705:	learn: 0.2471712	total: 5.36s	remaining: 2.23s
706:	learn: 0.2468853	total: 5.36s	remaining: 2.22s
707:	learn: 0.2468273	total: 5.37s	remaining: 2.21s
708:	learn: 0.2466807	total: 5.38s	remaining: 2.21s
709:	learn: 0.2464644	total: 5.39s	remaining: 2.2s
710:	learn: 0.2464315	total: 5.39s	remaining: 2.19s
711:	learn: 0.2463582	total: 5.4s	remaining: 2.18s
712:	learn: 0.2462405	total: 5.41s	remaining: 2.18s
713:	learn: 0.246

869:	learn: 0.2299607	total: 6.48s	remaining: 968ms
870:	learn: 0.2298927	total: 6.49s	remaining: 961ms
871:	learn: 0.2298195	total: 6.5s	remaining: 954ms
872:	learn: 0.2297999	total: 6.52s	remaining: 949ms
873:	learn: 0.2296259	total: 6.54s	remaining: 944ms
874:	learn: 0.2295263	total: 6.57s	remaining: 938ms
875:	learn: 0.2293510	total: 6.58s	remaining: 932ms
876:	learn: 0.2292746	total: 6.6s	remaining: 926ms
877:	learn: 0.2292010	total: 6.62s	remaining: 920ms
878:	learn: 0.2290853	total: 6.64s	remaining: 914ms
879:	learn: 0.2288590	total: 6.66s	remaining: 908ms
880:	learn: 0.2287849	total: 6.67s	remaining: 902ms
881:	learn: 0.2287634	total: 6.71s	remaining: 898ms
882:	learn: 0.2287262	total: 6.73s	remaining: 891ms
883:	learn: 0.2286679	total: 6.74s	remaining: 884ms
884:	learn: 0.2285487	total: 6.75s	remaining: 877ms
885:	learn: 0.2285116	total: 6.75s	remaining: 869ms
886:	learn: 0.2282029	total: 6.76s	remaining: 861ms
887:	learn: 0.2281895	total: 6.77s	remaining: 853ms
888:	learn: 0.

In [38]:
roc_auc_score(y_valid, clf_CTB.predict_proba(valid_df)[:, 1])

0.86640316205533596

## LogisticRegression

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
clf_LR = LogisticRegression(solver = 'liblinear', C = 1, random_state=17)

In [ ]:
clf_LR.fit(X_train, y_train)

In [ ]:
y_pred = clf_LR.predict(X_valid)

In [ ]:
print(accuracy_score(y_valid, y_pred))

# Export to csv

In [ ]:
ids = data_test['PassengerId']
predictions = clf.predict(data_test.drop('PassengerId', axis=1))


output = pd.DataFrame({ 'PassengerId' : ids, 'Survived': predictions })
# output.to_csv('titanic-predictions.csv', index = False)
output.head()